In [1]:
# Dependencies
import os
import csv
import json
import time
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from config import api_key
from citipy import citipy
import matplotlib.pyplot as plt

output_dir = os.path.join("Images","")

In [4]:
## Create latitudes and cities lists variables and initialize them as empty lists
cities = []
# Generate a list of cities using a random function up to 1500 in size for scanning
# Note: larger number of cities can be obtained with larger size, however for this task only about 500 are required
latitude = np.random.uniform(low=-90.000, high=90.000, size=200)
longitude = np.random.uniform(low=-180.000, high=180.000, size=200)

latitudes_longitudes = zip(latitude, longitude)
count_cities_500 = 0
for latitude_longitude in latitudes_longitudes:
    city = citipy.nearest_city(latitude_longitude[0], latitude_longitude[1]).city_name
    # If city is not in cities, then append the city to the cities' list. Limit the city list to 500
    if city not in cities:
        if count_cities_500 < 5:
            cities.append(city)
            count_cities_500 += 1
        else:
            break
    else:
        continue
print(f"First five are listed here: {cities[:5]}...")
print(f"There are {len(cities)} cities in the list.")

First five are listed here: ['georgetown', 'atuona', 'salalah', 'aykhal', 'tiksi']...
There are 5 cities in the list.


In [6]:
# url for weather api
units = 'english'
url = "http://api.openweathermap.org/data/2.5/weather?q="
city_5_list = cities[:5]
temp_list = []
for city in city_5_list:
    try:
        query_test_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
        print(query_test_url)
        response = requests.get(query_test_url).json()
        #print(response)
        temp_list.append(response["main"]["temp"])
        print(temp_list)
    except KeyError:
        print(f"City '{city.capitalize()}' was not found.")

http://api.openweathermap.org/data/2.5/weather?q=georgetown&appid=8d3fef753a916acc8df61a629cda8e70
[299.51]
http://api.openweathermap.org/data/2.5/weather?q=atuona&appid=8d3fef753a916acc8df61a629cda8e70
[299.51, 300.19]
http://api.openweathermap.org/data/2.5/weather?q=salalah&appid=8d3fef753a916acc8df61a629cda8e70
[299.51, 300.19, 304.15]
http://api.openweathermap.org/data/2.5/weather?q=aykhal&appid=8d3fef753a916acc8df61a629cda8e70
[299.51, 300.19, 304.15, 282.15]
http://api.openweathermap.org/data/2.5/weather?q=tiksi&appid=8d3fef753a916acc8df61a629cda8e70
[299.51, 300.19, 304.15, 282.15, 279.32]


In [ ]:
# README - Your first requirement is 
    # to create a series of scatter plots to showcase the following relationships:
# * Temperature (F) vs. Latitude
# * Humidity (%) vs. Latitude
# * Cloudiness (%) vs. Latitude
# * Wind Speed (mph) vs. Latitude

# Create an starter df
df = pd.DataFrame({'Temperature (F)': })

In [ ]:
temperaturesF = []
latitudes = []
longitudes = []
humiditiesPerc = []
cloudinessPerc = []
windspeedsMPH = []
max_temperatures = []
countries = []
dates = []
city_two = []

# Create data array from cities list
city_weather = []
store_index  = []
store_cityName = []
store_city_not_found = []
count_city_not_found = 0  # count number of cities not found

# transform cities into datframe for the progress bar
cities_df = pd.DataFrame(cities, columns=['City'])

# Run API calls to get the data for scatter plots as defined in parameters above:

# # set initial count quantities for organization
count_one = 0
set_one = 1

# loops for creating dataframe columns
for city in cities:
    
    try:
        url4 = (f"{url}{city}&appid={api_key}")
        print(url4)
        response = requests.get(url4).json()
        temperaturesF.append(response['main']['temp'])
    except Exception:
        print("City not found. Skipping...")

#     cloudinessPerc.append(response['clouds']['all'])
#     countries.append(response['sys']['country'])
#     dates.append(response['dt'])
#     humiditiesPerc.append(response['main']['humidity'])
#     latitudes.append(response['coord']['lat'])
#     longitudes.append(response['coord']['lon'])
#     max_temperatures.append(response['main']['temp_max'])
#     windspeedsMPH.append(response['wind']['speed'])
#     if count_one > 48:
#         count_one = 1
#         set_one += 1
#         city_two.append(city)
#     else:
#         count_one += 1
#         city_two.append(city)
#     print(f"Processing Record {count_one} of Set {set_one} | {city}")
#     except Exception:
# print("City not found. Skipping...")
# print("------------------------------\nData Retrieval Complete\n------------------------------")

In [ ]:
temp_df = pd.DataFrame({'Cities': (cities),
                        'Temperature, DegF': temperaturesF})
temp_df.head()